In [84]:
import datetime
import datetime as dt
first_time = datetime.datetime.utcnow()
dt.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S") 

'2022-10-26 03:36:26'

In [85]:
import pandas as pd
import sqlalchemy 
from sqlalchemy import create_engine
import pytz
# from urllib.parse import urlencode, quote
import requests

In [86]:
# MySQL Connection
MYSQL_ADDRESS = '######' 
MYSQL_PORT = '######' 
MYSQL_USERNAME = '######' 
MYSQL_PASSWORD = '######' 
MYSQL_DBNAME = '######'  
mysql_str = ('mysql+pymysql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=MYSQL_USERNAME, password=MYSQL_PASSWORD, ipaddress=MYSQL_ADDRESS, port=MYSQL_PORT,dbname=MYSQL_DBNAME))
# Create the connection
cnx_wh = create_engine(mysql_str)


In [87]:
# PostgreSQL Connection
POSTGRES_ADDRESS = '######' 
POSTGRES_PORT = '######' 
POSTGRES_USERNAME = '######' 
POSTGRES_PASSWORD = '######' 
POSTGRES_DBNAME = '######' 
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT,dbname=POSTGRES_DBNAME))
cnx_wh2 = create_engine(postgres_str)
conn2 = cnx_wh2.connect()
cron_activity_e = create_engine(postgres_str)
cron_activity = cron_activity_e.connect()

In [88]:
with cron_activity.connect() as con:
    con.execute("UPDATE cron_activity SET status = 'Running' where table_name = 'transactions_activity_warehouse' ;")

In [89]:
load_date_cron = pd.read_csv('load_date_update.csv')
ld = load_date_cron['date'].iloc[0]
ld = '"'+ld+'"'
ld

'"2022-10-26 03:34:32"'

In [90]:
save_date_cron = dt.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S") 

In [91]:
df = pd.read_sql_query('''
SELECT
	*
FROM
	transactions_activity_master 
WHERE updated_at >= {} 
'''.format(ld), cnx_wh)

In [ ]:
df.head()

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 42 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id                        25 non-null     int64         
 1   uuid                      25 non-null     object        
 2   program_category_id       25 non-null     int64         
 3   program_user_id           25 non-null     int64         
 4   user_id                   25 non-null     int64         
 5   promoter_id               0 non-null      object        
 6   program_id                25 non-null     int64         
 7   payment_method_id         25 non-null     int64         
 8   custom_user_name          4 non-null      object        
 9   transaction_number        25 non-null     object        
 10  transid_corez             24 non-null     object        
 11  quantity                  25 non-null     int64         
 12  base_amount             

== Insert to WH == 

In [95]:
POSTGRES_ADDRESS = '134.209.105.199' 
POSTGRES_PORT = '15482'
POSTGRES_USERNAME = 'odoo' 
POSTGRES_PASSWORD = 'erpdonolCintaAllah123!'
POSTGRES_DBNAME = 'erp-wh.cinte.id'
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT,dbname=POSTGRES_DBNAME))
cnx_wh2 = create_engine(postgres_str)
conn2 = cnx_wh2.connect()

In [96]:
# delete same transactions_id
meta = sqlalchemy.MetaData()

# Map the Inventory table in your database to a SQLAlchemy object
table_wh = sqlalchemy.Table('transactions_activity_warehouse', meta, autoload=True, autoload_with=cnx_wh2)

condf = df.apply(lambda row: sqlalchemy.and_(table_wh.c['transaction_id'] == row['transaction_id']), axis=1)
cond = sqlalchemy.or_(*condf)

# condf

# Define and execute the DELETE
delete = table_wh.delete().where(cond)
with cnx_wh2.connect() as conn2:
    conn2.execute(delete)


In [97]:
# insert data and data updated
cnx_wh2 = create_engine(postgres_str)
conn2 = cnx_wh2.connect()
df.to_sql('transactions_activity_warehouse', con=conn2, if_exists='append',index=False, method='multi')

25

In [100]:
#save date
sv = pd.DataFrame({
    'date':[save_date_cron]
})
sv.to_csv('load_date_update.csv')
sv

,date
0,2022-10-26 03:36:29


In [101]:
# processing_time
later_time = datetime.datetime.utcnow()
difference = later_time - first_time
seconds_in_day = 24 * 60 * 60

In [102]:
x_time = divmod(difference.days * seconds_in_day + difference.seconds, 60)
print(" processing_time, {}:{} seconds".format(x_time[0],x_time[1]))

 processing_time, 0:19 seconds


In [103]:
with cron_activity.connect() as con:
    con.execute("UPDATE cron_activity SET status = 'Done', last_processing_time = '{}:{}', updated_at = '{}' where table_name = 'transactions_activity_warehouse' ;".format(x_time[0],x_time[1],save_date_cron))